In [1]:
%pylab inline
from tqdm import tqdm_notebook
import os.path

from rd_model import RazzleDazzleGame, Position
from rd_bots import PositionBot, PositionNet, IdiotBot
from slab import get_current_filename, get_next_filename
from slab.datamanagement import SlabFile

# from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import random
import numpy as np


Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/IPython/qt.py:13: ShimWarning: The `IPython.qt` package has been deprecated since IPython 4.0. You should import from qtconsole instead.
  "You should import from qtconsole instead.", ShimWarning)


Warning could not load Chase AWG dll, check that dll located at 'C:\_Lib\python\slab\instruments\awg\chase\dax22000_lib_DLL32.dll'
Warning could not load LDA labbrick dll, check that dll located at 'C:\_Lib\python\slab\instruments\labbrick\VNX_atten.dll'
Warning could not load LMS labbrick dll, check that dll located at 'C:\_Lib\python\slab\instruments\labbrick\vnx_fmsynth.dll'
Warning could not load LPS labbrick dll, check that dll located at 'C:\_Lib\python\slab\instruments\labbrick\VNX_dps.dll'
Could not load labbrick
Could not load Dymo USB Scale
DSFIT could not import guiqwt


In [2]:
# Need to fix this, its a terrible algorithm
def itershuffle(iterable, bufsize=1000):
    """Shuffle an iterator. This works by holding `bufsize` items back
    and yielding them sometime later. This is NOT 100% random, proved or anything."""
    iterable = iter(iterable)
    buf = []
    try:
        while True:
            for i in xrange(random.randint(1, bufsize - len(buf))):
                buf.append(iterable.next())
            random.shuffle(buf)
            for i in xrange(random.randint(1, bufsize)):
                if buf:
                    yield buf.pop()
                else:
                    break
    except StopIteration:
        random.shuffle(buf)
        while buf:
            yield buf.pop()
        raise StopIteration


def generate_games(bot1, bot2, games, fname=None):
    """Generate lots of random games and save to text file"""
    for ii in range(games):
        game = RazzleDazzleGame(bot1, bot2)

        game.play()
        if fname is not None:
            with open(fname, 'a') as f:
                f.write(str(game) + '\n')
        yield str(game)


def get_game_data(game_str):
    if game_str.strip()[-1] == '8':  # get value
        v = 1
    else:
        v = -1
    pos = Position(score=v)
    white_move = True
    for ms in game_str.split(','):
        m = pos.parse_move(ms)
        if not white_move:
            m = pos.rotate_move(m)

        yield [pos.state.flatten(), pos.get_piece_on_square(m[0]), m[-1], v]
        pos=pos.move(m)
        pos.rotate()


def correct_predictions(output,value):
    return np.sum((1-torch.floor(torch.abs(value-output))).data.numpy())

def train_loader(bot1, bot2, games, training_fraction=1.0, fname=None):
    dataset = []
    for game_str in generate_games(bot1, bot2, games):
        one_game=list(get_game_data(game_str))
        for g in one_game[-20:]:
            dataset.append(g)
    random.shuffle(dataset)
    dataset = np.transpose(dataset[:int(len(dataset) * training_fraction)])

    return dataset

def generate_idiot_data(datapath, games, test_size):
    bot1=IdiotBot('i1')
    bot2=IdiotBot('i2')
    
    # Generate random legal game data
    for g in tqdm_notebook(range(games)):
        game = RazzleDazzleGame(bot1, bot2)
        game.play()
        
        with open(os.path.join(datapath,'idiot_games.txt'),'a') as f:
            f.write(str(game)+"\n")

    wins=[]
    losses=[]
    with open(os.path.join(datapath,'idiot_games.txt'),'r') as f:
        for game_str in tqdm_notebook(f):
            one_game=list(get_game_data(game_str))
            for g in one_game[-10:]:
                if g[-1]>0:
                    wins.append(g)
                else:
                    losses.append(g)

    num_scores=min(len(wins),len(losses))
    test_dataset=wins[:test_size]+losses[:test_size]
    dataset=wins[test_size:num_scores]+losses[test_size:num_scores]
    
    random.shuffle(dataset)
    random.shuffle(test_dataset)
    dataset = np.transpose(dataset)
    test_dataset = np.transpose(test_dataset)
    positions=np.vstack(dataset[0])
    test_positions=np.vstack(test_dataset[0])
    scores=np.array(dataset[-1],dtype=float)
    test_scores=np.array(test_dataset[-1],dtype=float)

    np.save(os.path.join(datapath,'idiot_positions.npy'),positions)
    np.save(os.path.join(datapath,'idiot_scores.npy'),scores)
    np.save(os.path.join(datapath,'test_positions.npy'),test_positions)
    np.save(os.path.join(datapath,'test_scores.npy'),test_scores)

In [10]:
#Generate Idiot Data
games=10000
bot1=IdiotBot()
bot2=IdiotBot()

fname='data/idiot_games.txt'

# Generate random legal game data
for g in tqdm_notebook(range(games)):
    game = RazzleDazzleGame(bot1, bot2)
    game.play()

    with open(fname,'a') as f:
        f.write(str(game)+"\n")

In [4]:
#generate idiot end games
fname='data/idiot_games.txt'
wins=[]
losses=[]
for game_str in tqdm_notebook(open(fname,'r')):
    one_game=list(get_game_data(game_str))
    for g in one_game[-10:]:
        if g[-1]>0:
            wins.append(g)
        else:
            losses.append(g)

num_scores=min(len(wins),len(losses))
test_size=4000
test_dataset=wins[:test_size]+losses[:test_size]
dataset=wins[test_size:num_scores]+losses[test_size:num_scores]
random.shuffle(dataset)
random.shuffle(test_dataset)
dataset = np.transpose(dataset)
test_dataset = np.transpose(test_dataset)
positions=np.vstack(dataset[0])
test_positions=np.vstack(test_dataset[0])
scores=np.array(dataset[-1],dtype=float)
test_scores=np.array(test_dataset[-1],dtype=float)

np.save('data/idiot_positions.npy',positions)
np.save('data/idiot_scores.npy',scores)
np.save('data/test_positions.npy',test_positions)
np.save('data/test_scores.npy',test_scores)

In [ ]:
mean(scores)

In [7]:
#Train idiot with idiot data
positions=np.load('data/idiot_positions.npy')
scores=np.load('data/idiot_scores.npy')
test_positions=np.load('data/test_positions.npy')
test_scores=np.load('data/test_scores.npy')

args = {}
args['batch_size'] = 200
args['cuda'] = False
args['epochs'] = 25
args['lr'] = 0.01
args['momentum'] = 0.5
args['log_interval'] = 10

epochs=args['epochs']
datapath="data/"
name="test"

fname = get_current_filename(datapath=datapath, prefix=name, suffix='.bot')

bot1 = PositionBot(name)
model = bot1.posnet


loss_fn=nn.MSELoss()

if args['cuda']:
    model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])


#test data/value
test_data  = Variable (torch.from_numpy(test_positions).float())
test_value = Variable (torch.from_numpy(test_scores).float())


for epoch in range(epochs):
    model.train()
    total=0
    correct=0
    for ii in range(len(scores)/args['batch_size']):
        data  = Variable (torch.from_numpy(positions[ii*args['batch_size']:(ii+1)*args['batch_size']]).float())
        value = Variable (torch.from_numpy(scores[ii*args['batch_size']:(ii+1)*args['batch_size']]).float())
    
        if args['cuda']:
            pass
            data, value = data.cuda(), value.cuda()
        optimizer.zero_grad()
        output = model(data)
        #hist(output.data.numpy(),bins=30)
        loss = loss_fn(output, value)
        loss.backward()
        optimizer.step()

        correct+=correct_predictions(output,value)
        total+=len(value)

    print "Train: %d/%d\t = %.5f%% correct \tLoss: %.6f\t%d/%d" % (correct,len(scores),correct*100./len(scores),loss.data[0],epoch+1,epochs)
    
    model.eval()
    test_output=model(test_data)
    test_loss=loss_fn(test_output,test_value)
    correct=correct_predictions(test_output,test_value)
    total=len(test_value)    
    print "Test:  %d/%d\t = %.5f%% correct\tLoss: %.6f" % (correct,total,correct*100./total,test_loss.data[0])
    torch.save(model.state_dict(), 'data/test.bot')
    

Failed to load: test.bot
Loading new PositionNet model
Train: 9974/20140	 = 49.52334% correct 	Loss: 0.999997	1/25
Test:  3971/8000	 = 49.63750% correct	Loss: 0.999947
Train: 10156/20140	 = 50.42701% correct 	Loss: 0.999486	2/25
Test:  4047/8000	 = 50.58750% correct	Loss: 0.999716
Train: 10286/20140	 = 51.07249% correct 	Loss: 0.998935	3/25
Test:  4171/8000	 = 52.13750% correct	Loss: 0.999471
Train: 10390/20140	 = 51.58888% correct 	Loss: 0.998332	4/25
Test:  4213/8000	 = 52.66250% correct	Loss: 0.999207
Train: 10560/20140	 = 52.43297% correct 	Loss: 0.997594	5/25
Test:  4265/8000	 = 53.31250% correct	Loss: 0.998882
Train: 10767/20140	 = 53.46077% correct 	Loss: 0.996617	6/25
Test:  4336/8000	 = 54.20000% correct	Loss: 0.998442
Train: 11003/20140	 = 54.63257% correct 	Loss: 0.995330	7/25
Test:  4373/8000	 = 54.66250% correct	Loss: 0.997849
Train: 11279/20140	 = 56.00298% correct 	Loss: 0.993602	8/25
Test:  4405/8000	 = 55.06250% correct	Loss: 0.997035
Train: 11618/20140	 = 57.68620% co

In [ ]:
hist(test_output.data.numpy(),bins=30);

In [8]:
bot1 = IdiotBot()
bot2 = PositionBot('test',path='data/')
bot3 = PositionBot('position',path='data/')
bot4 = PositionBot('position2',path='data/')



#bot1.posnet.load_state_dict(torch.load('data/test.bot'))

score=0
game_length=[]
for ii in tqdm_notebook(range(100)):
    game=RazzleDazzleGame(bot1,bot2)
    game.play()
    score+=game.winner
    game_length.append(len(game))

print score
print mean(game_length)

Failed to load: data/position.bot
Loading new PositionNet model
Failed to load: data/position2.bot
Loading new PositionNet model



-20
158.16


In [9]:
#Generate PositionBot Data
games=1000
bot1=PositionBot('position',path='data/')
bot2=PositionBot('position',path='data/')

fname='data/pbot2_games.txt'

# Generate random legal game data
for g in tqdm_notebook(range(games)):
    game = RazzleDazzleGame(bot1, bot2)
    game.play()

    with open(fname,'a') as f:
        f.write(str(game)+"\n")

Failed to load: data/position.bot
Loading new PositionNet model
Failed to load: data/position.bot
Loading new PositionNet model


KeyboardInterrupt: 

In [ ]:
#generate position end games
fname='data/pbot2_games.txt'
moves_from_end=20
wins=[]
losses=[]
for game_str in tqdm_notebook(open(fname,'r')):
    one_game=list(get_game_data(game_str))
    for g in one_game[-moves_from_end:]:
        if g[-1]>0:
            wins.append(g)
        else:
            losses.append(g)

num_scores=min(len(wins),len(losses))
test_size=2000
test_dataset=wins[:test_size]+losses[:test_size]
dataset=wins[test_size:num_scores]+losses[test_size:num_scores]
random.shuffle(dataset)
random.shuffle(test_dataset)
dataset = np.transpose(dataset)
test_dataset = np.transpose(test_dataset)
positions=np.vstack(dataset[0])
test_positions=np.vstack(test_dataset[0])
scores=np.array(dataset[-1],dtype=float)
test_scores=np.array(test_dataset[-1],dtype=float)

np.save('data/pbot2_positions.npy',positions)
np.save('data/pbot2_scores.npy',scores)
np.save('data/pbot2_test_positions.npy',test_positions)
np.save('data/pbot2_test_scores.npy',test_scores)

In [ ]:
#Train idiot with idiot data
positions=np.load('data/pbot2_positions.npy')
scores=np.load('data/pbot2_scores.npy')
test_positions=np.load('data/pbot2_test_positions.npy')
test_scores=np.load('data/pbot2_test_scores.npy')

args = {}
args['batch_size'] = 200
args['cuda'] = False
args['epochs'] = 25
args['lr'] = 0.001
args['momentum'] = 0.5
args['log_interval'] = 10

epochs=args['epochs']
datapath="data/"
name="position"

bot1 = PositionBot("test",path=datapath)
model = bot1.posnet


loss_fn=nn.MSELoss()

if args['cuda']:
    model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])


#test data/value
test_data  = Variable (torch.from_numpy(test_positions).float())
test_value = Variable (torch.from_numpy(test_scores).float())


for epoch in range(epochs):
    model.train()
    total=0
    correct=0
    for ii in range(len(scores)/args['batch_size']):
        data  = Variable (torch.from_numpy(positions[ii*args['batch_size']:(ii+1)*args['batch_size']]).float())
        value = Variable (torch.from_numpy(scores[ii*args['batch_size']:(ii+1)*args['batch_size']]).float())
    
        if args['cuda']:
            pass
            data, value = data.cuda(), value.cuda()
        optimizer.zero_grad()
        output = model(data)
        #hist(output.data.numpy(),bins=30)
        loss = loss_fn(output, value)
        loss.backward()
        optimizer.step()

        correct+=correct_predictions(output,value)
        total+=len(value)

    print "Train: %d/%d\t = %.5f%% correct \tLoss: %.6f\t%d/%d" % (correct,len(scores),correct*100./len(scores),loss.data[0],epoch+1,epochs)
    
    model.eval()
    test_output=model(test_data)
    test_loss=loss_fn(test_output,test_value)
    correct=correct_predictions(test_output,test_value)
    total=len(test_value)    
    print "Test:  %d/%d\t = %.5f%% correct\tLoss: %.6f" % (correct,total,correct*100./total,test_loss.data[0])
    torch.save(model.state_dict(), 'data/position2.bot')